<h1 align='center'>  Classification of Multiple Classes. </h1>

<p> &nbsp;&nbsp; Sometimes Neural Networks have to predict multiple classes, More then one class solution. There are 2 ways to solve this problem 
<ol>
    <li> Multi Label Classification </li>
    <li> Multi Class Classification </li>
</ol>

<p>&nbsp; 1. Multi Label Classification </br>
&nbsp;&nbsp;&nbsp; In this approach we basically have multiple outputs for multiple classes and get output as 1 or 0 for each neurons output. It invovles one hot encoding process of the dataset </p>

<p>&nbsp; 2. Multi Class Classification </br>
&nbsp;&nbsp;&nbsp; In this approach we use just the single output neuron to predict different classess itself. This involves label encoding process of dataset </p>

<img width="1200" height="800" src='https://www.section.io/engineering-education/multi-label-classification-with-scikit-multilearn/label-powerset.png'>

<p> Let's Work on Multi Class Classification first after which we can move to multi label classification </br>
We Can work with the <b><i> IRIS FLOWER DATASET </i></b>

<h5 align='center'> Dataset: </h5>
<img src='https://miro.medium.com/max/638/0*2c7voFri9cIXGrc4'>

<h3 align='center'> 0. Installing Dependencies </h3>

In [ ]:
!pip install tensorflow
!pip install numpy 
!pip install pandas 
!pip install -U scikit-learn

<h3 align='center'> 1. Importing Dependenceis </h3>

<p> We are going to import all the dependencies for this project. The Dependencies are: </p>
<ul>
    <li> Pandas </li>
    <li> Numpy </li>
    <li>tensorflow </li>
    <li> sklearn </li>
</ul>

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.model_selection import train_test_split

<h3 align='center'> 2. Handling Dataset </h3>
<p> &nbsp;&nbsp;&nbsp; We are going to handle the dataset in following ways: </p>
<ul>
    <li> Importing Dataset </li>
    <li> Label encoding </li>
    <li> One hot encoding </li>
</ul>

In [ ]:
def read_dataset(path):
    data=pd.read_csv(path)
    return data 

def get_class_dict(dataset):
    class_list=list(set(dataset['species'].to_list()))
    class_dct={}
    for i in range(len(class_list)):
        class_dct[class_list[i]]=i
    print(class_dct)

dataset=read_dataset('IRIS.csv')
classes_list=get_class_dict(dataset)